Our task is to design an IR system on the given dataset which can be found [here](https://drive.google.com/drive/folders/1h7AzOoPrbaEx2ApcGq-MO22quMcOi40H?usp=share_link. "link to dataset").

# STEP 1: dataset input
reading the files from the dataset. In order to do this step we use the OS library and list all the files available in the directory.

In [1]:
import os

#Path to directory
directory_path = "./docs"

#Getting a list of files in the directory
file_names = os.listdir(directory_path)

#DEBUG: checking the list
print(*file_names,sep='\n')

Jerry Decided To Buy a Gun.txt
Rentals at the Oceanside Community.txt
Gasoline Prices Hit Record High.txt
Cloning Pets.txt
Crazy Housing Prices.txt
Man Injured at Fast Food Place.txt
A Festival of Books.txt
Food Fight Erupted in Prison.txt
Better To Be Unlucky.txt
Sara Went Shopping.txt
Freeway Chase Ends at Newsstand.txt
Trees Are a Threat.txt
A Murder-Suicide.txt
Happy and Unhappy Renters.txt
Pulling Out Nine Tons of Trash.txt


Now that we have all the names, we can start by reading the content of every file:

In [3]:
#list of document content
file_content = []

for i in range(len(file_names)):
    file_names[i] = directory_path+"/"+file_names[i]
    #with open(directory_path+"/"+file_name, 'r', encoding='cp1252') as file:
    #    file_content.append(file.readlines())
    #    #print(file_content[-1])
    

# STEP 1.5: blocked sort-based indexing
We are going to implement the BSBI algorithm as defined in the slides. In order to do so, we define our blocks and then load the documents of each block. After making the inverted index of each block, we merge them all in one universal inverted index.

In [4]:
import os
import json
from collections import defaultdict

block_size = 3

def encode_gamma(num):
    # Get binary representation, excluding the leading '0b'
    binary_repr = bin(num + 1)[3:]
    gamma_code = '0' * (len(binary_repr) - 1) + binary_repr
    return gamma_code

def write_block_to_disk(sorted_terms, in_memory_index, block_num):
    # Write the in-memory index for a block to disk
    block_filename = f'block_{block_num}.json'
    with open(block_filename, 'w') as file:
        encoded_index = {term: [encode_gamma(gap) for gap in gaps] for term, gaps in in_memory_index.items()}
        json.dump({'sorted_terms': sorted_terms, 'index': encoded_index}, file)

def merge_blocks_on_disk():
    # Merge the indices from all blocks
    merged_index = defaultdict(list)
    block_file_names = [file for file in os.listdir() if file.startswith('block_')]

    for block_file in block_file_names:
        with open(block_file, 'r') as file:
            block_data = json.load(file)
            for term in block_data['sorted_terms']:
                gaps = [int(gap, 2) for gap in block_data['index'][term]]
                merged_index[term].extend(gaps)

    return merged_index

def write_merged_index_to_disk(merged_index):
    # Write the merged index to disk
    with open('inverted_index.json', 'w') as file:
        encoded_index = {term: [encode_gamma(gap) for gap in gaps] for term, gaps in merged_index.items()}
        json.dump(encoded_index, file)

def BSBI(block_size):

    # Create File Blocks
    blocks = [file_names[i:i + block_size] for i in range(0, len(file_names), block_size)]

    # Block Building and In-Memory Sorting
    block_content = []
    for block in blocks:
        in_memory_index = defaultdict(list)
        prev_doc_id = 0

        block_content.append([])
        for doc_path in block:
            with open(doc_path, 'r', encoding='cp1252') as doc_file:
                doc_content = doc_file.readlines()
                block_content[-1].append(doc_content)

blocks = []
block_content = []
BSBI(block_size)


# STEP 2: tokenizing
Now that we have the text in our program we should start by tokenizing the text.
In order to do this we're going to use the nltk library in python and to make our job easier we're going to tokenize words separated by Space, Comma, and dash.

Examples:
1. I live ...
2. Student, jason, ...
3. 30-year-old

In [5]:
from nltk.tokenize import RegexpTokenizer

# A list for tokenized texts
block_file_content_tokenized = []

# Define a regular expression pattern to match words separated by Space, Comma, and Dash
#pattern = r'\w+|\$[\d\.]+'
pattern = r'\d{1,4}(?:,\d{3})*(?:\.\d+)?|\w+'
#pattern = r'[A-Za-z0-9]+.*[A-Za-z0-9]+.*([+-]?(?=\.\d|\d)(?:\d+)?(?:\.?\d*))(?:[Ee]([+-]?\d+))?.*([0-9]+(,[0-9]+)+)'

for i in range(len(blocks)):
    #Change all input data into tokens
    block_file_content_tokenized.append([])
    for file in block_content[i]:
        # Use regex_tokenize with the defined pattern
        block_file_content_tokenized[i].append(RegexpTokenizer(pattern).tokenize(file[0]))
        print(block_file_content_tokenized[i][-1])


# STEP 3: lowercasing
We now want to lowercase our tokens:

In [6]:
for k in range(len(blocks)):
    #lowercasing each token individually
    for i in range(len(block_file_content_tokenized[k])):
        for j in range(len(block_file_content_tokenized[k][i])):
            block_file_content_tokenized[k][i][j] = block_file_content_tokenized[k][i][j].lower()
        print(block_file_content_tokenized[k][i])


# STEP 4: stopword removal
Now we want to remove the stopwords present in our text. To do this task we use the library of stopwords in nltk library. Please note that in order to be able to perform positional queries, we will not delete stopwords, just replace them with empty strings.

In [7]:
from nltk.corpus import stopwords

#Setting the stopword language
stop_words = set(stopwords.words("english"))
print(stop_words)

for k in range(len(blocks)):
    for i in range(len(block_file_content_tokenized[k])):
        j = 0
        #print(file_content_tokenized[i])
        while j < len(block_file_content_tokenized[k][i]):
            if block_file_content_tokenized[k][i][j] in stop_words:
                #print("got here with ", file_content_tokenized[i][j])
                #file_content_tokenized[i] = file_content_tokenized[i][:j]+file_content_tokenized[i][j+1:]
                block_file_content_tokenized[k][i][j] = ""
            else:
                j += 1
        print(block_file_content_tokenized[k][i])



{'after', 'had', 'on', 'myself', "shan't", "you're", 're', 'doesn', 'each', 'once', 'will', 'there', 'should', 'weren', 'ain', 'd', 'are', 'have', 'very', 'an', 'you', 'do', 'why', 'me', 'into', 'below', 'shan', "haven't", 'those', 'shouldn', 'no', 'up', "doesn't", 'about', "hasn't", 'too', 'isn', 'under', 'other', 'my', 'more', 'down', 's', 'itself', 'it', 'these', "you'll", 'because', 'so', 'if', 'she', 'were', 'can', 'the', 'while', 'haven', 'does', 'mightn', "don't", 'we', 'and', 'any', 'm', 'how', 'nor', 'hers', 'wouldn', 'yourselves', 'what', 'our', 'its', 'from', 'ma', 'against', 'ours', "wouldn't", 'wasn', 'then', 'just', "weren't", 'all', 'i', 'during', "mustn't", 'before', 'ourselves', 'here', 'as', 'not', 'most', 'who', 'same', "mightn't", "you've", "didn't", "aren't", 'in', 'than', 'aren', 'but', 'when', 'they', 'your', 'of', 'his', "isn't", "should've", 'o', 'hasn', 'further', "hadn't", 'did', 'this', "she's", 'where', 'theirs', 'be', 'hadn', 'now', 'out', "won't", "that'l

# STEP 6.5: BSBI Sort and Completion
Now that we have processed our documents in the blocks, let's just do the sorting and writing

In [9]:
for i, block in enumerate(blocks):
    in_memory_index = defaultdict(list)
    prev_doc_id = 0
    for j in range(len(block_file_content_tokenized[i])):
            for term in block_file_content_tokenized[i][j]:
                doc_id = int(os.path.basename(file_names[j]))
                gap = doc_id - prev_doc_id
                in_memory_index[term].append(gap)
                prev_doc_id = doc_id

    # Sort the terms in memory
    sorted_terms = sorted(in_memory_index.keys())

    # Write to Disk
    write_block_to_disk(sorted_terms, in_memory_index, i)

# Merge All
merged_index = merge_blocks_on_disk()

# Write the Merged Data to Disk
write_merged_index_to_disk(merged_index)